In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
from keras.preprocessing.text import Tokenizer
from sklearn.metrics.pairwise import cosine_similarity
from keras.preprocessing.sequence import pad_sequences
from difflib import SequenceMatcher

In [ ]:
!python -m spacy download it_core_news_lg

2023-11-28 22:34:05.773183: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-28 22:34:05.773252: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-28 22:34:05.773313: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-28 22:34:06.891737: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.9/567.9 MB 1.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_lg')


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/cleaned (1).csv', delimiter = ';')
df.head()

,Unnamed: 0,From,To,Subject,Date,Body,Protocollo,OBJECT_ID,Committente,Nome e Cognome,...,Descrizione Categoria 2,Categoria 3,Descrizione Categoria 3,Canale,Numero Sistema Esterno,Stato_x,Descr.Stato,Mercato di provenienza,Email_count,Body_cleaned
0,38,silvio@nerobutto.it,no-reply.clienti.commerciale@dolomitienergia.it,R: Mandato per addebito diretto SEPA C00001219...,30/11/2022,\r\n\r\n\r\n\r\nin allegato invio modulo firma...,2022_312622,4.002535e+09,20349452.0,LINA STEFANI,...,SEPA,C285,Prestazione,E-mail,OTX,E0041,Chiusa,Tutela,3,allegato invio modulo firmato geom silvio volt...
1,40,silvio@nerobutto.it,no-reply.clienti.commerciale@dolomitienergia.it,R: Mandato per addebito diretto SEPA C00001219...,30/11/2022,\r\n\r\n\r\n\r\nin allegato invio modulo firma...,2022_312624,4.002535e+09,20349452.0,LINA STEFANI,...,SEPA,C299,Richiesta informazioni,E-mail,OTX,E0041,Chiusa,Tutela,3,allegato invio modulo firmato geom silvio volt...
2,42,amministrazione@impresamazzoleni.it,no-reply.clienti.commerciale@dolomitienergia.it,richiesta chiusura fornitura di corrente per i...,30/11/2022,\r\n\r\n\r\ncontratto n.101498129 del 01/11/20...,2022_312626,4.002535e+09,20290420.0,NaN,...,Invio Modulistica,C280,Prestazione,E-mail,OTX,E0041,Chiusa,Tutela,4,contratto codice cliente buongiorno presente r...
3,43,cianfroccaservice.info@gmail.com,no-reply.clienti.commerciale@dolomitienergia.it,Re: Invio offerta PUN-TA PREMIUM LIBERO,30/11/2022,"\r\n\r\n\r\n\r\n\r\nbuona sera,\r\nin allegato...",2022_312627,4.002535e+09,20360798.0,NaN,...,Offerte firmate,C305,Richiesta informazioni,E-mail,OTX,E0041,Chiusa,Tutela,6,buona sera allegato modulo offerta compilato d...
4,44,birimbi@hotmail.it,clienti.commerciale@dolomitienergia.it,Re: Attivazione fornitura sul mercato libero,30/11/2022,\r\n\r\n\r\n\r\n\r\nbuongiorno come da accordi...,2022_312628,4.002535e+09,20309652.0,NaN,...,Modifica anagrafica,C303,Richiesta informazioni,E-mail,OTX,E0041,Chiusa,Tutela,1,buongiorno accordi telefonici invio pec associ...


In [ ]:
df = df[~df['From'].str.contains("@dolomitienergia")]
df.head()

,Unnamed: 0,From,To,Subject,Date,Body,Protocollo,OBJECT_ID,Committente,Nome e Cognome,...,Descrizione Categoria 2,Categoria 3,Descrizione Categoria 3,Canale,Numero Sistema Esterno,Stato_x,Descr.Stato,Mercato di provenienza,Email_count,Body_cleaned
0,38,silvio@nerobutto.it,no-reply.clienti.commerciale@dolomitienergia.it,R: Mandato per addebito diretto SEPA C00001219...,30/11/2022,\r\n\r\n\r\n\r\nin allegato invio modulo firma...,2022_312622,4.002535e+09,20349452.0,LINA STEFANI,...,SEPA,C285,Prestazione,E-mail,OTX,E0041,Chiusa,Tutela,3,allegato invio modulo firmato geom silvio volt...
1,40,silvio@nerobutto.it,no-reply.clienti.commerciale@dolomitienergia.it,R: Mandato per addebito diretto SEPA C00001219...,30/11/2022,\r\n\r\n\r\n\r\nin allegato invio modulo firma...,2022_312624,4.002535e+09,20349452.0,LINA STEFANI,...,SEPA,C299,Richiesta informazioni,E-mail,OTX,E0041,Chiusa,Tutela,3,allegato invio modulo firmato geom silvio volt...
2,42,amministrazione@impresamazzoleni.it,no-reply.clienti.commerciale@dolomitienergia.it,richiesta chiusura fornitura di corrente per i...,30/11/2022,\r\n\r\n\r\ncontratto n.101498129 del 01/11/20...,2022_312626,4.002535e+09,20290420.0,NaN,...,Invio Modulistica,C280,Prestazione,E-mail,OTX,E0041,Chiusa,Tutela,4,contratto codice cliente buongiorno presente r...
3,43,cianfroccaservice.info@gmail.com,no-reply.clienti.commerciale@dolomitienergia.it,Re: Invio offerta PUN-TA PREMIUM LIBERO,30/11/2022,"\r\n\r\n\r\n\r\n\r\nbuona sera,\r\nin allegato...",2022_312627,4.002535e+09,20360798.0,NaN,...,Offerte firmate,C305,Richiesta informazioni,E-mail,OTX,E0041,Chiusa,Tutela,6,buona sera allegato modulo offerta compilato d...
4,44,birimbi@hotmail.it,clienti.commerciale@dolomitienergia.it,Re: Attivazione fornitura sul mercato libero,30/11/2022,\r\n\r\n\r\n\r\n\r\nbuongiorno come da accordi...,2022_312628,4.002535e+09,20309652.0,NaN,...,Modifica anagrafica,C303,Richiesta informazioni,E-mail,OTX,E0041,Chiusa,Tutela,1,buongiorno accordi telefonici invio pec associ...


In [ ]:
df = df.dropna(subset=['Body_cleaned']).reset_index(drop=True)

In [ ]:
nlp = spacy.load("it_core_news_lg")
def preprocess_email_body(text):
    doc = nlp(text)
    tokens = [token.text for token in doc if not token.is_stop and token.is_alpha]
    return " ".join(tokens)

In [ ]:
df["Body_cleaned"] = df["Body"].apply(preprocess_email_body)
df['Body_cleaned'].head()

15    allegato invio modulo firmato geom silvio volt...
17    allegato invio modulo firmato geom silvio volt...
19    contratto codice cliente buongiorno presente r...
20    buona sera allegato modulo offerta compilato d...
21    buongiorno accordi telefonici invio pec associ...
Name: Body_cleaned, dtype: object

In [ ]:
#df.to_csv('cleaned.csv', index=False, sep=';')

In [ ]:
'''result_df = pd.DataFrame()

body_similarity_dict_all = {}





In [ ]:
import pandas as pd

result_df = pd.DataFrame()

portion_size = 10000  # Size of each sequential portion
portion_index = 1  # Process only the first portion

# Calculate the start and end index for the selected portion
start_index = portion_index * portion_size
end_index = (portion_index + 1) * portion_size

# Select a sequential portion of 100,000 rows from the DataFrame
portion_df = df.iloc[start_index:end_index]

# Iterate over 5 random selections within the portion
for _ in range(1):
    # Get a random index from within the portion
    random_email_index = np.random.choice(portion_df.index)

    # Get the corresponding row from the DataFrame
    random_email = portion_df.loc[[random_email_index]].reset_index(drop=True)

    result = comparison_7(random_email, portion_df)

    # Append the result to the final DataFrame
    result_df = pd.concat([result_df, result], ignore_index=True)

    print(f"\nIteration {_ + 1}: Original Emails")
    print(random_email['From'])
    print(f"\nIteration {_ + 1}: Number of Elements in result DataFrame: {len(result)}")

# Display the final DataFrame with duplicates
print("\nFinal Result DataFrame:")
result_df.head(10)


In [ ]:
def similarity_score(str1, str2):
    return SequenceMatcher(None, str1, str2).ratio()

def extract_username(email):
    # Extract the username before the '@' sign
    return email.split('@')[0]

def comparison_7(new_email, df, from_similarity_threshold=0.5):
    sender = new_email['From']
    receiver = new_email['To']
    body = new_email['Body_cleaned']

    # Filter DataFrame based on sender and receiver
    similar_sender = df[(df['From'] == sender.iloc[0]) & (df['To'] == receiver.iloc[0])].reset_index(drop=True)

    # Filter DataFrame based on sender (different receiver)
    similar_sender_diff_receiver = df[(df['From'] == sender.iloc[0]) & (df['To'] != receiver.iloc[0])].reset_index(drop=True)

    # Tokenize and pad sequences for body only
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(df['Body_cleaned'])
    sequences = tokenizer.texts_to_sequences(df['Body_cleaned'])
    max_length = max(len(seq) for seq in sequences)
    padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

    # Tokenize and pad the new email body
    tokenizer.fit_on_texts([body.iloc[0]])
    new_sequence = tokenizer.texts_to_sequences([body.iloc[0]])
    new_padded_sequence = pad_sequences(new_sequence, maxlen=max_length, padding='post')

    # Calculate cosine similarity for body only
    body_similarity_scores = cosine_similarity(new_padded_sequence, padded_sequences)

    simi = body_similarity_scores[0]
    sorted_indices = np.argsort(simi)[::-1]
    sorted_simi = simi[sorted_indices]

    # Add top similar indices to the set
    top_similar_indices_set = set(sorted_indices[:5])

    # Access the original emails using DataFrame indexing
    top_similar_emails = df.iloc[list(top_similar_indices_set)].reset_index(drop=True)

    # Print some debugging information
    print("Sorted Body Similarity Scores:")
    print(sorted_simi[:5])
    print("Top Similar Indices:")
    print(top_similar_indices_set)
    print("Original Emails:")
    print(top_similar_emails[['From']])

    # Extract username from sender and apply similarity_score
    extracted_sender_username = extract_username(sender.iloc[0])
    from_email_similarity_scores = df['From'].apply(lambda x: similarity_score(extract_username(x), extracted_sender_username))

    # Get indices with similarity above the threshold
    similar_from_indices = from_email_similarity_scores[from_email_similarity_scores > from_similarity_threshold].index

    # Filter top_similar_emails based on similar "From" emails
    top_similar_emails = top_similar_emails[top_similar_emails.index.isin(similar_from_indices)].reset_index(drop=True)

    # Sort top_similar_emails based on the original indices
    top_similar_emails = top_similar_emails.reindex(sorted(top_similar_emails.index))

    # Create a new column "similarity_score" in top_similar_emails and associate similarity scores
    top_similar_emails['similarity_score'] = sorted_simi[:len(top_similar_emails)]



    merged_df = pd.concat([similar_sender, similar_sender_diff_receiver, top_similar_emails], ignore_index=True).drop_duplicates()
    return merged_df